# Asset Portfolio Management using Deep Reinforcement Learning
----

## 2.0 Download the Data for the Analysis
---
* Data is Downloaded from Yahoo Finance using an API in the FinRL Library
* The data is downloaded and the cleaned data are saved in csv files for use

---


### 2.1 Import Relevant Packages

In [1]:
import pandas as pd
import numpy as np
import my_config
import matplotlib.pylab as plt

import yfinance as yf
from pandas_datareader import data as pdr

%matplotlib inline

### 2.2 Create Folders

In [2]:
import os
if not os.path.exists("./" + my_config.DATA_SAVE_DIR):
    os.makedirs("./" + my_config.DATA_SAVE_DIR)
if not os.path.exists("./" + my_config.TRAINED_MODEL_DIR):
    os.makedirs("./" + my_config.TRAINED_MODEL_DIR)
if not os.path.exists("./" + my_config.TENSORBOARD_LOG_DIR):
    os.makedirs("./" + my_config.TENSORBOARD_LOG_DIR)
if not os.path.exists("./" + my_config.RESULTS_DIR):
    os.makedirs("./" + my_config.RESULTS_DIR)

### 2.3 Download the stock data

In [3]:
ticker_list = my_config.TICKERS

In [4]:
# öljy data
oil_ticker = "BZ=F"  # Yahoo Finance ticker for Brent Crude Oil Futures
oildata = yf.download(oil_ticker, start="2014-12-08", end="2025-1-1", interval="1d")
oildata = oildata.reset_index()
oildata = oildata[["Date", "Close"]]
oildata["oil_change"] = ((oildata["Close"] / oildata["Close"].shift(1)) - 1)
oildata.head()

[*********************100%***********************]  1 of 1 completed


Price,Date,Close,oil_change
Ticker,,BZ=F,
0,2014-12-08,66.190002,NaN
1,2014-12-09,66.839996,0.009820
2,2014-12-10,64.239998,-0.038899
3,2014-12-11,63.680000,-0.008717
4,2014-12-12,61.849998,-0.028737


In [5]:
if isinstance(oildata.columns, pd.MultiIndex):  # Check if multi-level columns exist
    oildata.columns = oildata.columns.droplevel(0)  # Flatten the columns

oildatanew = oildata.drop(columns=oildata.columns[1])  # Drop the middle column
oildatanew.columns = ["Date", "oil_change"]  # Rename the columns
oildatanew.head()

,Date,oil_change
0,2014-12-08,NaN
1,2014-12-09,0.009820
2,2014-12-10,-0.038899
3,2014-12-11,-0.008717
4,2014-12-12,-0.028737


In [6]:
#10 vuoden bondin data
yield_ticker = "^TNX"  # Yahoo Finance ticker for 10-Year Treasury Yield
yielddata = yf.download(yield_ticker, start="2014-12-08", end="2025-1-1", interval="1d")
yielddata = yielddata.reset_index()
yielddata = yielddata[["Date", "Close"]]
yielddata["yield_change"] = ((yielddata["Close"] / yielddata["Close"].shift(1)) - 1)
yielddata.head()

[*********************100%***********************]  1 of 1 completed


Price,Date,Close,yield_change
Ticker,,^TNX,
0,2014-12-08,2.257,NaN
1,2014-12-09,2.220,-0.016393
2,2014-12-10,2.169,-0.022973
3,2014-12-11,2.178,0.004149
4,2014-12-12,2.103,-0.034435


In [7]:
if isinstance(yielddata.columns, pd.MultiIndex):  # Check if multi-level columns exist
    yielddata.columns = yielddata.columns.droplevel(0)
yielddatanew = yielddata.drop(columns=yielddata.columns[1])
yielddatanew.columns = ["Date", "yield_change"]
yielddatanew

,Date,yield_change
0,2014-12-08,NaN
1,2014-12-09,-0.016393
2,2014-12-10,-0.022973
3,2014-12-11,0.004149
4,2014-12-12,-0.034435
...,...,...
2527,2024-12-24,-0.001739
2528,2024-12-26,-0.002614
2529,2024-12-27,0.008736
2530,2024-12-30,-0.016021


In [8]:
#yhdistely
merged_macrodata = pd.merge(oildatanew, yielddatanew, on="Date", how="inner")
merged_macrodata.head()

,Date,oil_change,yield_change
0,2014-12-08,NaN,NaN
1,2014-12-09,0.009820,-0.016393
2,2014-12-10,-0.038899,-0.022973
3,2014-12-11,-0.008717,0.004149
4,2014-12-12,-0.028737,-0.034435


In [9]:
merged_macrodata.to_csv('datasets/merged_data.csv', index=False)

In [10]:
import yfinance as yf

# Assuming `ticker_list` contains the list of Dow Jones tickers
ticker = ticker_list[0]  # First ticker as an example

In [11]:
# osakkeiden lataus
data = yf.download(ticker_list, start="2014-12-08", end="2025-1-1", group_by='ticker')

data = data.stack(level=0).reset_index()

print(data.columns)
data.columns = ['Date', 'Ticker', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']

data = data.drop(columns=['Close'])

print(data.head())

[*********************100%***********************]  10 of 10 completed
C:\Users\Evert\AppData\Local\Temp\ipykernel_9636\3155022218.py:5: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  data = data.stack(level=0).reset_index()


Index(['Date', 'Ticker', 'Open', 'High', 'Low', 'Close', 'Adj Close',
       'Volume'],
      dtype='object', name='Price')
        Date     Ticker       Open       High        Low  Adj Close  Volume
0 2014-12-08   BOREO.HE   4.100000   4.250000   4.100000   3.412212    1253
1 2014-12-08   ELISA.HE  23.790001  23.950001  23.719999  15.474872  227944
2 2014-12-08   ICP1V.HE   0.332111   0.332111   0.332111   0.332111    3914
3 2014-12-08   MEKKO.HE   1.860000   1.870000   1.850000   1.356999    7815
4 2014-12-08  NDA-FI.HE  10.190000  10.210000  10.100000   5.240574  618254


In [26]:
data.head()

,Date,Ticker,Open,High,Low,Adj Close,Volume
0,2014-12-08,BOREO,4.100000,4.250000,4.100000,3.412212,1253
1,2014-12-08,ELISA,23.790001,23.950001,23.719999,15.474872,227944
2,2014-12-08,ICP1V,0.332111,0.332111,0.332111,0.332111,3914
3,2014-12-08,MEKKO,1.860000,1.870000,1.850000,1.356999,7815
4,2014-12-08,NDA-FI,10.190000,10.210000,10.100000,5.240574,618254


In [29]:
data['Ticker'] = data['Ticker'].str.replace('.HE', '', regex=False)

In [30]:
data.head(10)

,Date,Ticker,Open,High,Low,Adj Close,Volume
0,2014-12-08,BOREO,4.100000,4.250000,4.100000,3.412212,1253
1,2014-12-08,ELISA,23.790001,23.950001,23.719999,15.474872,227944
2,2014-12-08,ICP1V,0.332111,0.332111,0.332111,0.332111,3914
3,2014-12-08,MEKKO,1.860000,1.870000,1.850000,1.356999,7815
4,2014-12-08,NDA-FI,10.190000,10.210000,10.100000,5.240574,618254
5,2014-12-08,NESTE,6.673333,6.673333,6.560000,5.028741,2226561
6,2014-12-08,OLVAS,22.000000,22.000000,21.299999,15.715649,9510
7,2014-12-08,SAMPO,35.448936,35.502712,35.260712,22.012814,572235
8,2014-12-08,UPM,13.780000,13.840000,13.580000,8.666770,1330070
9,2014-12-08,YIT,5.320000,5.330000,5.210000,3.473339,632675


In [31]:
unique_dates = data[["Date"]].drop_duplicates()  # Get unique dates from stock data
aligned_macrodata = pd.merge(unique_dates, merged_macrodata, on="Date", how="left")  # Merge with macrodata
aligned_macrodata

,Date,oil_change,yield_change
0,2014-12-08,NaN,NaN
1,2014-12-09,0.009820,-0.016393
2,2014-12-10,-0.038899,-0.022973
3,2014-12-11,-0.008717,0.004149
4,2014-12-12,-0.028737,-0.034435
...,...,...,...
2523,2024-12-19,-0.006949,0.016911
2524,2024-12-20,0.000823,-0.010066
2525,2024-12-23,-0.004250,0.016578
2526,2024-12-27,0.012421,0.008736


In [32]:
expanded_macrodata = aligned_macrodata.loc[aligned_macrodata.index.repeat(10)].reset_index(drop=True)

merged_data = data.copy()
merged_data["oil_change"] = expanded_macrodata["oil_change"]
merged_data["yield_change"] = expanded_macrodata["yield_change"]

merged_data  # Show the first 20 rows to check the alignment


,Date,Ticker,Open,High,Low,Adj Close,Volume,oil_change,yield_change
0,2014-12-08,BOREO,4.100000,4.250000,4.100000,3.412212,1253,NaN,NaN
1,2014-12-08,ELISA,23.790001,23.950001,23.719999,15.474872,227944,NaN,NaN
2,2014-12-08,ICP1V,0.332111,0.332111,0.332111,0.332111,3914,NaN,NaN
3,2014-12-08,MEKKO,1.860000,1.870000,1.850000,1.356999,7815,NaN,NaN
4,2014-12-08,NDA-FI,10.190000,10.210000,10.100000,5.240574,618254,NaN,NaN
...,...,...,...,...,...,...,...,...,...
25275,2024-12-30,NESTE,11.785000,12.125000,11.760000,12.125000,2236292,0.002966,-0.016021
25276,2024-12-30,OLVAS,29.400000,29.400000,28.950001,29.200001,9640,0.002966,-0.016021
25277,2024-12-30,SAMPO,39.070000,39.380001,38.959999,39.380001,600560,0.002966,-0.016021
25278,2024-12-30,UPM,26.500000,26.780001,26.430000,26.559999,1045678,0.002966,-0.016021


In [33]:
merged_data.isna().sum()

Date              0
Ticker            0
Open              0
High              0
Low               0
Adj Close         0
Volume            0
oil_change      690
yield_change    690
dtype: int64

In [34]:
merged_data=merged_data.dropna()
merged_data

,Date,Ticker,Open,High,Low,Adj Close,Volume,oil_change,yield_change
10,2014-12-09,BOREO,4.250000,4.250000,4.250000,3.412212,0,0.009820,-0.016393
11,2014-12-09,ELISA,23.610001,23.820000,23.440001,15.383613,389383,0.009820,-0.016393
12,2014-12-09,ICP1V,0.321893,0.321893,0.311674,0.311674,2388,0.009820,-0.016393
13,2014-12-09,MEKKO,1.850000,1.850000,1.806000,1.334994,11685,0.009820,-0.016393
14,2014-12-09,NDA-FI,10.080000,10.150000,9.965000,5.168291,1273926,0.009820,-0.016393
...,...,...,...,...,...,...,...,...,...
25275,2024-12-30,NESTE,11.785000,12.125000,11.760000,12.125000,2236292,0.002966,-0.016021
25276,2024-12-30,OLVAS,29.400000,29.400000,28.950001,29.200001,9640,0.002966,-0.016021
25277,2024-12-30,SAMPO,39.070000,39.380001,38.959999,39.380001,600560,0.002966,-0.016021
25278,2024-12-30,UPM,26.500000,26.780001,26.430000,26.559999,1045678,0.002966,-0.016021


In [35]:
print(merged_data.columns)

Index(['Date', 'Ticker', 'Open', 'High', 'Low', 'Adj Close', 'Volume',
       'oil_change', 'yield_change'],
      dtype='object')


In [36]:
merged_data.isna().sum()

Date            0
Ticker          0
Open            0
High            0
Low             0
Adj Close       0
Volume          0
oil_change      0
yield_change    0
dtype: int64

In [37]:
merged_data.columns=['date','tic','open','high','low','close','volume', 'oil_chg', 'yield_chg']

In [38]:
merged_data.columns

Index(['date', 'tic', 'open', 'high', 'low', 'close', 'volume', 'oil_chg',
       'yield_chg'],
      dtype='object')

In [39]:
no_datasets = []
for i in ticker_list:
    no_data_points = merged_data[merged_data['tic']==i].shape[0]
    no_datasets.append((i,no_data_points))
    data_points_df = pd.DataFrame(no_datasets)

In [40]:
data_points_df

,0,1
0,NESTE.HE,0
1,UPM.HE,0
2,ICP1V.HE,0
3,YIT.HE,0
4,OLVAS.HE,0
5,MEKKO.HE,0
6,BOREO.HE,0
7,NDA-FI.HE,0
8,SAMPO.HE,0
9,ELISA.HE,0


### 2.5 Save the Data to csv

In [41]:
merged_data.to_csv('datasets/merged_data.csv', index=False)

### 2.6 Create a Dataset for the Close Prices

In [42]:
df_prices = pd.read_csv('./datasets/merged_data.csv')

In [43]:
df_prices = df_prices.reset_index().set_index(['tic', 'date']).sort_index()

In [44]:
tic_list = list(set([i for i,j in df_prices.index]))
tic_list

['ELISA',
 'OLVAS',
 'ICP1V',
 'SAMPO',
 'NESTE',
 'UPM',
 'NDA-FI',
 'YIT',
 'MEKKO',
 'BOREO']

In [45]:
df_prices = df_prices.reset_index().set_index(['tic', 'date']).sort_index()

df_close = pd.DataFrame()

for ticker in tic_list:
    series = df_prices.xs(ticker).close
    df_close[ticker] = series

df_close = df_close.reset_index()
df_close = df_close[['date'] + sorted(df_close.columns.drop('date'))]

In [46]:
df_close

,date,BOREO,ELISA,ICP1V,MEKKO,NDA-FI,NESTE,OLVAS,SAMPO,UPM,YIT
0,2014-12-09,3.412212,15.383613,0.311674,1.334994,5.168291,4.881361,15.422766,21.767355,8.450420,3.470012
1,2014-12-10,3.412212,15.455315,0.316783,1.334994,5.178615,4.937263,15.551346,21.717148,8.494963,3.443397
2,2014-12-11,3.412212,15.572650,0.332111,1.334994,5.150218,4.949968,15.929948,21.912397,8.456783,3.420108
3,2014-12-12,3.412212,15.188060,0.332111,1.334994,4.987580,4.700944,16.179976,21.438225,8.259521,3.356896
4,2014-12-15,3.412212,14.692656,0.301455,1.342329,4.858502,4.739060,16.058537,21.025414,8.074987,3.114028
...,...,...,...,...,...,...,...,...,...,...,...
2454,2024-12-19,10.000000,41.459999,10.090000,11.520000,10.465000,11.455000,28.900000,38.810001,25.709999,2.366000
2455,2024-12-20,10.400000,41.560001,10.250000,11.880000,10.245000,11.225000,28.850000,39.009998,25.660000,2.384000
2456,2024-12-23,10.250000,41.400002,10.200000,12.100000,10.300000,11.155000,29.000000,38.990002,25.719999,2.442000
2457,2024-12-27,10.150000,42.119999,10.190000,11.860000,10.450000,11.785000,29.400000,39.130001,26.570000,2.498000


In [51]:
df_close.describe().T

,count,mean,std,min,25%,50%,75%,max
BOREO,2459.0,19.787446,18.152219,3.058948,7.250435,10.400000,31.983365,113.941208
ELISA,2459.0,35.235421,10.427952,14.431914,24.994849,38.531757,44.445675,53.480583
ICP1V,2459.0,5.342502,5.274789,0.275908,1.072977,2.669566,9.935000,21.750000
MEKKO,2459.0,6.262457,4.540590,0.929349,1.630798,4.991809,10.358660,16.726713
NDA-FI,2459.0,6.791747,1.956508,3.325954,5.214315,6.211135,8.225153,11.775000
NESTE,2459.0,24.709848,14.093659,4.700944,10.349188,24.749454,37.635052,57.599873
OLVAS,2459.0,27.770864,7.753686,14.965585,21.805853,27.511234,31.283252,48.690899
SAMPO,2459.0,31.494782,5.947742,17.561613,26.878824,29.772694,37.346258,44.145020
UPM,2459.0,21.902892,6.963144,8.074987,16.649189,22.253923,27.980516,34.827030
YIT,2459.0,3.806938,1.049035,1.581000,3.004840,4.046261,4.475039,5.870115


In [52]:

df_close.to_csv('datasets/close_prices.csv', index=False)

In [62]:
df_close

,date,BOREO,ELISA,ICP1V,MEKKO,NDA-FI,NESTE,OLVAS,SAMPO,UPM,YIT
0,2014-12-09,3.412212,15.383613,0.311674,1.334994,5.168291,4.881361,15.422766,21.767355,8.450420,3.470012
1,2014-12-10,3.412212,15.455315,0.316783,1.334994,5.178615,4.937263,15.551346,21.717148,8.494963,3.443397
2,2014-12-11,3.412212,15.572650,0.332111,1.334994,5.150218,4.949968,15.929948,21.912397,8.456783,3.420108
3,2014-12-12,3.412212,15.188060,0.332111,1.334994,4.987580,4.700944,16.179976,21.438225,8.259521,3.356896
4,2014-12-15,3.412212,14.692656,0.301455,1.342329,4.858502,4.739060,16.058537,21.025414,8.074987,3.114028
...,...,...,...,...,...,...,...,...,...,...,...
2454,2024-12-19,10.000000,41.459999,10.090000,11.520000,10.465000,11.455000,28.900000,38.810001,25.709999,2.366000
2455,2024-12-20,10.400000,41.560001,10.250000,11.880000,10.245000,11.225000,28.850000,39.009998,25.660000,2.384000
2456,2024-12-23,10.250000,41.400002,10.200000,12.100000,10.300000,11.155000,29.000000,38.990002,25.719999,2.442000
2457,2024-12-27,10.150000,42.119999,10.190000,11.860000,10.450000,11.785000,29.400000,39.130001,26.570000,2.498000
